In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import uproot
import awkward
import mplhep as hep
import glob
hep.style.use("CMS")


Path_files = "/home/tomas/U1_LQ"


class RootTreeReader:

    """ 
    Read data from a ROOT TTree 
    Parameters:
    path : string
        Path to the ROOT file
    tree_name : string (default=Delphes)
        Name of the ROOT TTree
    Attributes:
    tree: Root TTree 
    """

    def __init__(self, path: str, tree_name: str = "Delphes"):
        self.tree = uproot.open(path)[tree_name]


    def get_branches(self, branches = ["MissingET.MET",
                                       "MissingET.Eta",
                                       "MissingET.Phi",
                                       "Jet.PT",
                                       "Jet.Eta",
                                       "Jet.Phi",
                                       "Jet.Mass",
                                       "Jet.TauTag",
                                       "Jet.BTag",
                                       "Jet_size"], max_elements=4):
        """
        returns a DataFrame with branches as features
        branches : array-like
          branches to load from the ROOT tree
        max_elements : int (default=4)
          maximum number of elements to load from jagged arrays
        """   
        self._max_elements = max_elements
        self._df = pd.DataFrame(index=range(self.tree.num_entries))

        for branch in branches:
            self._join_branch(branch)

        return self._set_columns_names(self._df)


    def _join_branch(self, branch):
        """joins a branch to self._df"""
        df = self.tree.arrays(branch, library="pd")

        if "." in branch:
            if len(df) > len(df.groupby(level=0).size()):
                self._add_jagged_branch(df, branch)
            else:
                self._add_branch(df, branch)
        else:
            self._add_branch(df, branch)


    def _add_branch(self, df, branch: str):
        """adds a non-jagged branch to self.df"""
        self._df[branch] = self.tree[branch].array(library="pd").values


    def _add_jagged_branch(self, df, branch):
        """adds a jagged branch to self.df"""
        df = df.unstack().iloc[:,:self._max_elements]
        df.columns = ["{0}{1}".format(branch, i) for i in range(self._max_elements)]
        self._df = self._df.join(df)

    @staticmethod
    def _set_columns_names(df):
        df.columns = df.columns.str.lower().str.replace(".","_")
        return df


def build_df(path):
    """
    Generates a Dataframe from the root in "path"
    """
    reader = RootTreeReader(path)
    df = reader.get_branches()
    df["n_b"]  =  reader.tree.arrays("Jet.BTag", library="pd").sum(level=0)
    df["n_tau"] = reader.tree.arrays("Jet.TauTag", library="pd").sum(level=0)
    return df


def tau_cut(df, val = 0):
    mask = (df.n_tau > val) & (df.jet_tau_index1 >= 0)
    return df.loc[mask]

def b_cut(df, val = 0):
    mask = (df.n_b > val) & (df.jet_b_index1 >= 0)
    return df.loc[mask]


#Cuts over the final Df

def pt_tau_cut(df, val = 25):
    # leading jets pt > 30 GeV
    mask = df.tau1_pT > val
    return df.loc[mask]


def pt_b_cut(df, val = 30):
    # leading jets pt > 30 GeV
    mask = df.b_pT > val
    return df.loc[mask]


def eta_tau_cut(df, val = 2.4):
    # leading jets eta < 2.4
    mask = np.abs(df.tau1_eta) < val
    return df.loc[mask]


def eta_b_cut(df, val = 2.1):
    # leading jets eta < 2.4
    mask = np.abs(df.b_eta) < val
    return df.loc[mask]


def phi_tau_cut(df, val= 2.0):
    # delta phi between met and tau > 2.0
    mask = np.abs(df.Delta_phi_Tau_Met) > val
    return df.loc[mask]


def et_met_cut(df, val = 200):
    # Met et greater than 200
    mask = df.met_Met > val
    return df.loc[mask]


def final_cuts(df, pt_tau = 25, pt_b = 20, eta_tau = 2.3, eta_b = 2.5, met = 150, del_phi = 1.0):
    """
    Returns a copy of the df filtered by different variables and different objects
    Parameters:
        df : A Pandas.Dataframe to be filtered. This DataFrame must have a series of columns named as 
             'tau_pT' ,'tau_eta' ,'tau_phi' ,'tau_mass', 'b_pT' ,'b_eta' ,'b_phi' ,
             'b_mass','met_Met' ,'met_Phi' ,'met_Eta', 'n_tau', 'n_b'.
        pt_tau : Minimun value for tau's p_T.
        pt_b : Minimun value for b's p_T.
        eta_tau : Minimun value for tau's eta.
        eta_b : Minimun value for b's eta.
        met : Minimun value for p_T^{miss}. 
        del_phi : Minimun value for absolute value of delta phi between tau and met.
    """
    cut_df = df.copy()
    cut_df = pt_tau_cut(cut_df, pt_tau)
    cut_df = pt_b_cut(cut_df, pt_b)
    cut_df = eta_tau_cut(cut_df, eta_tau)
    cut_df = phi_tau_cut(cut_df, del_phi)
    cut_df = eta_b_cut(cut_df, eta_b)
    cut_df = et_met_cut(cut_df, met)
    return cut_df





def branch_index(df, branch):
    """
    Adds a column with the index of the first jet tagged as branch to the df.

    Branch:
    takes values of "jet_btag" or "jet_tautag"

    Ex:
    branch_index(cut_df, "jet_tautag")
    Returns a df with a column with the first jet tagged as tau per ivent
    """
    branch_jets = df[[f"{branch}{i}" for i in range(4)]].copy()

    # events with branch jets
    branch_events1 = branch_jets[branch_jets.sum(axis=1) > 0]
    
    # events with more tah 1 branch jets
    branch_events2 = branch_jets[branch_jets.sum(axis=1) > 1]

    # index of first branch jet
    branch_index1 = branch_events1.apply(lambda x: x > 0).apply(lambda x: np.nonzero(x.values)[0][0], axis=1)
    
    # index of first branch jet
    branch_index2 = branch_events2.apply(lambda x: x > 0).apply(lambda x: np.nonzero(x.values)[0][1], axis=1)

    # index of non branch jets (set to nan)
    branch_nan = pd.DataFrame(index= branch_jets[branch_jets.sum(axis=1) == 0].index)

    # branch jet index
    df["{}_index1".format(branch).replace('tag','')] = pd.concat([branch_index1,branch_nan]).sort_index()
    df["{}_index2".format(branch).replace('tag','')] = pd.concat([branch_index2,branch_nan]).sort_index()

    return df , branch_index1, branch_index2

def DeltaPhi(row, col1 = 'tau1_phi', col2 = 'met_Phi'):
    """
    correction on azimuthal angle difference dphi
    """
    dphi = row[col1] - row[col2]
    if dphi >= np.pi: 
        dphi -= 2*np.pi
    if dphi < -np.pi:
        dphi += 2*np.pi

    return dphi


def m_Tot(row):  
    #Calculates TotalMass from 5.4  https://arxiv.org/pdf/1709.07242.pdf   
    pt1 = row['tau_pT']
    px1 = row['tau_pT'] * np.cos(row['tau_phi'])
    py1 = row['tau_pT'] * np.sin(row['tau_phi'])
    pt2 = row['b_pT']
    px2 = row['b_pT'] * np.cos(row['b_phi'])
    py2 = row['b_pT'] * np.sin(row['b_phi'])
    met_pt3 = row['met_Met']
    px3 = met_pt3 * np.cos(row['met_Phi'])
    py3 = met_pt3 * np.sin(row['met_Phi'])


    vec1 = np.array([px1 , py1])
    vec2 = np.array([px2 , py2])
    vec3 = np.array([px3 , py3])
    vect_t = vec1 + vec2 + vec3 
    vec_t2 = np.dot(vect_t, vect_t)
    sum_escal = (pt1 + pt2 + met_pt3) **2
    return (sum_escal - vec_t2  ) ** 0.5


def transverse_mass(tau_pt, met_et, deltaphi):
    #Calculates the transverse mass between tau (or any other jet) and the met
    return np.sqrt(2 * tau_pt * met_et * (1 - np.cos(deltaphi)))


def invariant_mass(obj1_pt, obj1_eta, obj2_pt, obj2_eta, deltaphi ):
    #Calculates the invariant mass for 2 different objects
    return np.sqrt(2 * obj1_pt * obj2_pt * (np.cosh(obj1_eta-obj2_eta) - np.cos(deltaphi)))


def Get_Pt_Eta_Phi_Tau_B(row):
    """
    Returns a row with a list of the Pt, Eta, Phi
    of the Taus and B's (in the respective order)
    Needs a column named Tau_b_Tuple.
    """
    if pd.isna(row['jet_tau_index2']):
        
        n_tau1 = int(row['jet_tau_index1'])
        n_tau2 = np.NaN
        n_b1 = int(row['jet_b_index1'])
        num_b = row['n_b']

        s = pd.Series(cols)
        s1 = list(s[s.astype(str).str[-1] == str(n_tau1)][:4])
        #s1_2 = list(s[s.astype(str).str[-1] == str(n_tau2)][:4])
        s2 = list(s[s.astype(str).str[-1] == str(n_b1)][:4])


        tau_pT1 = row[str(s1[0])] 
        tau_eta1 = row[str(s1[1])] 
        tau_phi1 = row[str(s1[2])] 
        tau_mass1 = row[str(s1[3])]

        tau_pT2 = np.NaN
        tau_eta2 = np.NaN 
        tau_phi2 = np.NaN 
        tau_mass2 = np.NaN

        b_pT = row[str(s2[0])] 
        b_eta = row[str(s2[1])] 
        b_phi = row[str(s2[2])] 
        b_mass = row[str(s2[3])]

        met_Met = row['missinget_met']
        met_Phi = row['missinget_phi']
        met_Eta = row['missinget_eta']
        
    else:
        n_tau1 = int(row['jet_tau_index1'])
        n_tau2 = int(row['jet_tau_index2'])
        n_b1 = int(row['jet_b_index1'])
        num_b = row['n_b']

        s = pd.Series(cols)
        s1 = list(s[s.astype(str).str[-1] == str(n_tau1)][:4])
        s1_2 = list(s[s.astype(str).str[-1] == str(n_tau2)][:4])
        s2 = list(s[s.astype(str).str[-1] == str(n_b1)][:4])


        tau_pT1 = row[str(s1[0])] 
        tau_eta1 = row[str(s1[1])] 
        tau_phi1 = row[str(s1[2])] 
        tau_mass1 = row[str(s1[3])]

        tau_pT2 = row[str(s1_2[0])] 
        tau_eta2 = row[str(s1_2[1])] 
        tau_phi2 = row[str(s1_2[2])] 
        tau_mass2 = row[str(s1_2[3])]

        b_pT = row[str(s2[0])] 
        b_eta = row[str(s2[1])] 
        b_phi = row[str(s2[2])] 
        b_mass = row[str(s2[3])]

        met_Met = row['missinget_met']
        met_Phi = row['missinget_phi']
        met_Eta = row['missinget_eta']

    return (tau_pT1, tau_eta1, tau_phi1, tau_mass1, 
            tau_pT2, tau_eta2, tau_phi2, tau_mass2, 
            b_pT, b_eta, b_phi, b_mass, 
            met_Met, met_Phi, met_Eta, n_tau1, num_b, n_b1)

def generate_data_b_tau_nu(df):
    """Returns a Dataframe with the information per event
    of the tau_jet and the missin energy.
    The index preserves the index from the original dataframe.
    Arguments:
        df :  dataframe generated with 
        get_braches("MissingET.MET","MissingET.Eta","MissingET.Phi","Jet.PT",
                    "Jet.Eta","Jet.Phi","Jet.Mass","Jet.TauTag","Jet.BTag","Jet_size")

    Also the dataframe must content a column named as n_tau as the number of taus per event
    Columns : 
        'tau_pT' ,'tau_eta' ,'tau_phi' ,'tau_mass', 'b_pT' ,'b_eta' ,'b_phi' ,
        'b_mass','met_Met' ,'met_Phi' ,'met_Eta', 'n_tau', 'n_b'.

    """
    cut_df, tau_index1, tau_index2  = branch_index(df, 'jet_tautag')
    cut_df, b_index1, b_index2 = branch_index(cut_df, 'jet_btag')
    #cut_df, tau_index = branch_index(df, 'jet_tautag')
    #cut_df, tau_index = branch_index(cut_df, 'jet_btag')
    cut_df = tau_cut(cut_df)
    cut_df = b_cut(cut_df)

    s = cut_df.apply(Get_Pt_Eta_Phi_Tau_B, axis = 1)
    Df = pd.DataFrame(s.to_list(), index = s.index, 
                      columns=['tau1_pT' ,
                               'tau1_eta' ,
                               'tau1_phi' ,
                               'tau1_mass', 
                               'tau2_pT', 
                               'tau2_eta', 
                               'tau2_phi', 
                               'tau2_mass',
                               'b_pT' ,
                               'b_eta' ,
                               'b_phi' ,
                               'b_mass',
                               'met_Met' ,
                               'met_Phi' ,
                               'met_Eta' ,
                               'n_tau1', 
                               'num_b', 'n_b1'])
  
    Df['Delta_phi_Tau_Met'] = Df.apply(DeltaPhi,axis = 1)
    Df['Delta_phi_B_Met'] = Df.apply(DeltaPhi,axis = 1, args=('b_phi', 'met_Phi'))
    Df['Delta_phi_Tau_B'] = Df.apply(DeltaPhi,axis = 1, args=('tau1_phi', 'b_phi'))
    Df = Df.copy()[Df.n_tau1 != Df.n_b1]
    return Df

In [2]:
lw = glob.glob("/cms/mc/Samples/SMWtaunu_btaunu_analysis/*.root")
lz = glob.glob("/cms/mc/MG5_aMC_v3_1_1/DY+jets/Events/*/*.root")

In [3]:
lw

['/cms/mc/Samples/SMWtaunu_btaunu_analysis/run_12.root',
 '/cms/mc/Samples/SMWtaunu_btaunu_analysis/run_03.root',
 '/cms/mc/Samples/SMWtaunu_btaunu_analysis/run_14.root',
 '/cms/mc/Samples/SMWtaunu_btaunu_analysis/run_04.root',
 '/cms/mc/Samples/SMWtaunu_btaunu_analysis/run_06.root',
 '/cms/mc/Samples/SMWtaunu_btaunu_analysis/run_11.root',
 '/cms/mc/Samples/SMWtaunu_btaunu_analysis/run_09.root',
 '/cms/mc/Samples/SMWtaunu_btaunu_analysis/run_08.root',
 '/cms/mc/Samples/SMWtaunu_btaunu_analysis/run_07.root',
 '/cms/mc/Samples/SMWtaunu_btaunu_analysis/run_13.root',
 '/cms/mc/Samples/SMWtaunu_btaunu_analysis/run_10.root',
 '/cms/mc/Samples/SMWtaunu_btaunu_analysis/run_05.root',
 '/cms/mc/Samples/SMWtaunu_btaunu_analysis/run_15.root']

In [3]:
l_dfz = []
l_z_e_mu = []
for path in lz:
    df_z = build_df(path)
    l_dfz.append(df_z)
    reader = RootTreeReader(path)
    df_e_mu = reader.get_branches(branches=["Electron.PT"], max_elements=1)
    try:
        mu = reader.get_branches(branches=["Muon.PT"], max_elements=1)
        df_e_mu['muon_pt'] = mu.iloc[:,0]
    except:
        print(f"No se pudo llenar con la rama de muones para la {path[-11::]}")
        df_e_mu['muon_pt'] = np.NaN
    l_z_e_mu.append(df_e_mu)
    

df_zpj = pd.concat([df for df in l_dfz],ignore_index= True)
df_zpj_e_mu = pd.concat([df for df in l_z_e_mu],ignore_index= True)
cols = df_zpj.columns
print(f"Se produjo Z +jets con {df_zpj.shape[0]}")

/tmp/ipykernel_1814853/3288760798.py:83: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df.columns = df.columns.str.lower().str.replace(".","_")
/tmp/ipykernel_1814853/3288760798.py:93: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  df["n_b"]  =  reader.tree.arrays("Jet.BTag", library="pd").sum(level=0)
/tmp/ipykernel_1814853/3288760798.py:94: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  df["n_tau"] = reader.tree.arrays("Jet.TauTag", library="pd").sum(level=0)


Se produjo Z +jets con 88419


In [4]:
l_dfw = []
l_w_e_mu = []
for path in lw:
    df_w = build_df(path)
    l_dfw.append(df_w)
    reader = RootTreeReader(path)
    df_e_mu = reader.get_branches(branches=["Electron.PT"], max_elements=1)
    try:
        mu = reader.get_branches(branches=["Muon.PT"], max_elements=1)
        df_e_mu['muon_pt'] = mu.iloc[:,0]
    except:
        print(f"No se pudo llenar con la rama de muones para la {path[-11::]}")
        df_e_mu['muon_pt'] = np.NaN
    l_w_e_mu.append(df_e_mu)
    

df_wpj = pd.concat([df for df in l_dfw],ignore_index= True)
df_wpj_e_mu = pd.concat([df for df in l_w_e_mu],ignore_index= True)
cols = df_wpj.columns
print(f"Se produjo W +jets con {df_wpj.shape[0]}")

/tmp/ipykernel_1814853/3288760798.py:83: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df.columns = df.columns.str.lower().str.replace(".","_")
/tmp/ipykernel_1814853/3288760798.py:93: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  df["n_b"]  =  reader.tree.arrays("Jet.BTag", library="pd").sum(level=0)
/tmp/ipykernel_1814853/3288760798.py:94: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  df["n_tau"] = reader.tree.arrays("Jet.TauTag", library="pd").sum(level=0)


No se pudo llenar con la rama de muones para la run_06.root
No se pudo llenar con la rama de muones para la run_15.root
Se produjo W +jets con 350495


In [6]:
Df_bg_wj = generate_data_b_tau_nu(df_wpj)
Df_bg_zj = generate_data_b_tau_nu(df_zpj)


In [7]:
Df_bg_wj['electron_pT'] = df_wpj_e_mu.loc[Df_bg_wj.index,'electron_pt0']
Df_bg_wj['muon_pT'] = df_wpj_e_mu.loc[Df_bg_wj.index,'muon_pt']
Df_bg_zj['electron_pT'] = df_zpj_e_mu.loc[Df_bg_zj.index,'electron_pt0']
Df_bg_zj['muon_pT'] = df_zpj_e_mu.loc[Df_bg_zj.index,'muon_pt']

Df_bg_wj.to_csv("/home/tomas/w+jets_e_mu_one_b.csv")
Df_bg_zj.to_csv("/home/tomas/z+jets_e_mu_one_b.csv")

In [8]:
opt = 250
odp = 1.5
omet = 200


Df_bgw_cut = final_cuts(Df_bg_wj, pt_tau = opt, pt_b=20, del_phi = odp, met = omet)
Df_bgz_cut = final_cuts(Df_bg_zj, pt_tau = opt, pt_b=20, del_phi = odp, met = omet)

Df_bgw_cut = Df_bgw_cut[Df_bgw_cut.num_b == 1]
Df_bgz_cut = Df_bgz_cut[Df_bgz_cut.num_b == 1]

Df_bgw_cut = Df_bgw_cut[(Df_bgw_cut.electron_pT < 15) | ((Df_bgw_cut.electron_pT.isna()))]
Df_bgw_cut = Df_bgw_cut[(Df_bgw_cut.muon_pT < 15)     | ((Df_bgw_cut.muon_pT.isna()))]
Df_bgz_cut = Df_bgz_cut[(Df_bgz_cut.electron_pT < 15) | ((Df_bgz_cut.electron_pT.isna()))]
Df_bgz_cut = Df_bgz_cut[(Df_bgz_cut.muon_pT < 15)     | ((Df_bgz_cut.muon_pT.isna()))]

Df_bgw_cut = Df_bgw_cut[(Df_bgw_cut.tau2_pT < 50) | ((Df_bgw_cut.tau2_pT.isna()))]
Df_bgw_cut = Df_bgw_cut[(np.abs(Df_bgw_cut.tau2_eta) > 2.3) | ((Df_bgw_cut.tau2_eta.isna()))]
Df_bgz_cut = Df_bgz_cut[(Df_bgz_cut.tau2_pT < 50) | ((Df_bgz_cut.tau2_pT.isna()))]
Df_bgz_cut = Df_bgz_cut[(np.abs(Df_bgz_cut.tau2_eta) > 2.3) | ((Df_bgz_cut.tau2_eta.isna()))]

Df_bgw_cut.shape, Df_bgz_cut.shape

((104, 23), (65, 23))